In [5]:
%load_ext google.cloud.bigquery

# Scripting

## A sequence of SQL statements, seperated by ;

In [6]:
%%bigquery

# Create table typical_trip

CREATE OR REPLACE TABLE
  dataflow-templates-327714.bigquery_examples.typical_trip AS
SELECT
  start_station_name,
  end_station_name,
  APPROX_QUANTILES(duration, 10)[
OFFSET
  (5)] AS typical_duration,
  COUNT(*) AS num_trips
FROM
  dataflow-templates-327714.bigquery_examples.cycle_hire
GROUP BY
  start_station_name,
  end_station_name;

# Create table unusual days

CREATE OR REPLACE TABLE
  dataflow-templates-327714.bigquery_examples.unusual_days AS
    SELECT EXTRACT(DATE FROM start_date) AS trip_date,
    APPROX_QUANTILES(duration / typical_duration, 10)[OFFSET(5)] AS ratio,
    COUNT(*) AS num_trips_on_day
    FROM dataflow-templates-327714.bigquery_examples.cycle_hire as hire,
    bigquery_examples.typical_trip AS trip
WHERE
    hire.start_station_name = trip.start_station_name
    AND hire.end_station_name = trip.end_station_name
    AND num_trips > 10;


Executing query with job ID: c47dd723-140f-4597-b1d7-f810e8388230
Query executing: 9.88s


ERROR:
 400 Query error: SELECT list expression references column start_date which is neither grouped nor aggregated at [22:30]

Location: EU
Job ID: c47dd723-140f-4597-b1d7-f810e8388230



## A simple script starts with variables

#### Use case: Find the return stations with longest duration rentals from waterloo

- Here we declare variables to:
   - Represent the pattern
   - Minimum number of trips threshold 
   - Array of strings to store intermediate data related to stations that fit the pattern
- **Variables can be any type supported by BigQuery**

In [7]:
%%bigquery

-- variables
DECLARE
  PATTERN STRING DEFAULT '%Waterloo%';
DECLARE
  stations ARRAY<STRING>;
DECLARE
  MIN_TRIPS_THRESH INT64 DEFAULT 100;

SET
  stations = (
  SELECT
    ARRAY_AGG(name)
  FROM
    dataflow-templates-327714.bigquery_examples.cycle_stations
  WHERE
    name LIKE PATTERN );
    
SELECT
  start_station_name,
  end_station_name,
  AVG(duration) AS avg_duration,
  COUNT(duration) AS num_trips
FROM
  dataflow-templates-327714.bigquery_examples.cycle_hire CROSS JOIN
  UNNEST(stations) AS station
WHERE
  start_station_name = station
GROUP BY
  start_station_name,
  end_station_name
HAVING
  num_trips > MIN_TRIPS_THRESH
ORDER BY
  avg_duration DESC
LIMIT
  5;

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  3.72rows/s]


,start_station_name,end_station_name,avg_duration,num_trips
0,"Lambeth North Station, Waterloo","Binfield Road, Stockwell",7204.000000,150
1,"Baylis Road, Waterloo","Binfield Road, Stockwell",6520.833333,288
2,"Lower Marsh, Waterloo","Caldwell Street, Stockwell",6420.685714,175
3,"Waterloo Station 2, Waterloo","Waterloo Station 1, Waterloo",5456.953125,256
4,"Waterloo Place, St. James's","Tower Gardens , Tower",4421.320755,212


## Looping

Scripting also supports control flows through IF conditions and a variety of looping primitives. Below example is a simple while loop as a function of the MIN_TRIPS_THRESH variable.

In [8]:
%%bigquery

-- variables
DECLARE
  PATTERN STRING DEFAULT '%Waterloo%';
DECLARE
  stations ARRAY<STRING>;
DECLARE
  MIN_TRIPS_THRESH INT64 DEFAULT 100;
SET
  stations = (
  SELECT
    ARRAY_AGG(name)
  FROM
    dataflow-templates-327714.bigquery_examples.cycle_stations
  WHERE
    name LIKE PATTERN );

WHILE MIN_TRIPS_THRESH < 1000 DO
SELECT
  start_station_name,
  end_station_name,
  AVG(duration) AS avg_duration,
  COUNT(duration) AS num_trips
FROM
  dataflow-templates-327714.bigquery_examples.cycle_hire,
  UNNEST(stations) AS station
WHERE
  start_station_name = station
GROUP BY
  start_station_name,
  end_station_name
HAVING
  num_trips > MIN_TRIPS_THRESH
ORDER BY
  avg_duration DESC
LIMIT
  5;

SET MIN_TRIPS_THRESH = MIN_TRIPS_THRESH * 2;
END WHILE;

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  3.72rows/s]


,start_station_name,end_station_name,avg_duration,num_trips
0,"Waterloo Station 1, Waterloo","Waterloo Station 1, Waterloo",3792.509091,825
1,"Waterloo Place, St. James's","Waterloo Place, St. James's",3071.813691,2834
2,"Waterloo Place, St. James's","Wellington Arch, Hyde Park",1452.331386,1883
3,"Waterloo Station 1, Waterloo","Tower Gardens , Tower",1298.056380,1348
4,"Waterloo Station 1, Waterloo","Brushfield Street, Liverpool Street",1233.096271,1153


### More primitive loops

In [9]:
%%bigquery

-- variable
DECLARE
  MIN_TRIPS_THRESH INT64 DEFAULT 100;
  
-- primitive (classic) loop
LOOP
IF MIN_TRIPS_THRESH >= 1000 THEN
    BREAK;
END IF;

SELECT MIN_TRIPS_THRESH;
SET MIN_TRIPS_THRESH = MIN_TRIPS_THRESH * 2;
END LOOP;

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.49s/rows]


,MIN_TRIPS_THRESH
0,800


## Exceptions

### Catching exceptions

In [10]:
%%bigquery

BEGIN
DECLARE
  stations ARRAY<INT64>;
SET
  stations = (
  SELECT
    ARRAY_AGG(CAST(name AS INT64)) names
  FROM
    dataflow-templates-327714.bigquery_examples.cycle_stations
  WHERE
    name LIKE '%Kings%'); EXCEPTION
    WHEN ERROR THEN SELECT @@error.message AS msg, @@error.stack_trace AS trace;
END
  ;

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.20s/rows]


,msg,trace
0,"Query error: Bad int64 value: Kingsway, Covent...","[{'line': 4, 'column': 1, 'filename': None, 'l..."


### Raising exceptions

In [11]:
%%bigquery

BEGIN
DECLARE
  stations ARRAY<STRING>;
SET
  stations = (
  SELECT
    ARRAY_AGG(name) names
  FROM
    dataflow-templates-327714.bigquery_examples.cycle_stations
  WHERE
    name LIKE '%potato%'); -- Replace with another value to trigger exception

IF ARRAY_LENGTH(stations) = 0 THEN
   RAISE USING MESSAGE = "No stations matched";
END IF;
  
   EXCEPTION
    WHEN ERROR THEN SELECT @@error.message AS msg, @@error.stack_trace AS trace;
END
  ;

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.19s/rows]


,msg,trace
0,No stations matched,"[{'line': 14, 'column': 4, 'filename': None, '..."


## Dynamic SQL

It is possible to create a string dynamically within a script and execute it using EXECUTE IMMEDIATE

In [14]:
%%bigquery

SELECT * FROM `bigquery-public-data.covid19_jhu_csse.confirmed_cases` WHERE country_region LIKE 'Canada' LIMIT 10;

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.70rows/s]


,province_state,country_region,latitude,longitude,location_geom,_1_22_20,_1_23_20,_1_24_20,_1_25_20,_1_26_20,...,_5_22_22,_5_23_22,_5_24_22,_5_25_22,_5_26_22,_5_27_22,_5_28_22,_5_29_22,_5_30_22,_5_31_22
0,Diamond Princess,Canada,0.0000,0.0000,POINT(0 0),0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,New Brunswick,Canada,46.5653,-66.4619,POINT(-66.4619 46.5653),0,0,0,0,0,...,64490,64490,64490,65258,65258,65258,65258,65258,65258,65258
2,Repatriated Travellers,Canada,NaN,NaN,None,0,0,0,0,0,...,13,13,13,13,13,13,13,13,13,13
3,Grand Princess,Canada,0.0000,0.0000,POINT(0 0),0,0,0,0,0,...,13,13,13,13,13,13,13,13,13,13
4,Newfoundland and Labrador,Canada,53.1355,-57.6604,POINT(-57.6604 53.1355),0,0,0,0,0,...,45409,45409,45409,45409,45775,45775,45775,45775,45775,45775
5,Northwest Territories,Canada,64.8255,-124.8457,POINT(-124.8457 64.8255),0,0,0,0,0,...,12026,12026,12046,12046,12046,12046,12046,12046,12089,12089
6,Nunavut,Canada,70.2998,-83.1076,POINT(-83.1076 70.2998),0,0,0,0,0,...,3531,3531,3531,3531,3531,3531,3531,3531,3531,3531
7,Nova Scotia,Canada,44.6820,-63.7443,POINT(-63.7443 44.682),0,0,0,0,0,...,94893,94893,94893,94893,94893,96477,96477,96477,96477,96477
8,Yukon,Canada,64.2823,-135.0000,POINT(-135 64.2823),0,0,0,0,0,...,4349,4349,4363,4366,4370,4378,4378,4378,4386,4387
9,Manitoba,Canada,53.7609,-98.8139,POINT(-98.8139 53.7609),0,0,0,0,0,...,143603,143603,143603,143603,144140,144140,144140,144140,144140,144140


### Generate and execute queries dynamically

In [15]:
%%bigquery

DECLARE
  col_0 STRING;
SET
  col_0 = '_5_18_20';

EXECUTE IMMEDIATE
  FORMAT("""
    SELECT country_region, province_state, %s AS cases_day0
    FROM `bigquery-public-data.covid19_jhu_csse.confirmed_cases`
    WHERE country_region LIKE 'Canada'
    ORDER BY cases_day0 DESC
    """, col_0)

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00, 10.32rows/s]


,country_region,province_state,cases_day0
0,Canada,Quebec,44258
1,Canada,Ontario,24033
2,Canada,Alberta,6683
3,Canada,British Columbia,2444
4,Canada,Nova Scotia,1043
5,Canada,Saskatchewan,592
6,Canada,Manitoba,290
7,Canada,Newfoundland and Labrador,260
8,Canada,New Brunswick,120
9,Canada,Prince Edward Island,27


### Generating views based on INFORMATION_SCHEMA

In [18]:
%%bigquery

FOR TBL IN (
 SELECT
   TABLE_NAME
 FROM
   `bq-info-schema-analysis.bqsap.INFORMATION_SCHEMA.TABLES`
 WHERE
   TABLE_TYPE='BASE TABLE') DO
BEGIN
 
DECLARE
 FIELDS_ARR ARRAY<STRING> DEFAULT ARRAY(
 SELECT
   CASE
     WHEN data_type = 'STRING' THEN FORMAT('REGEXP_REPLACE(%s, "\'" , "") as %s', column_name, column_name)
   ELSE
   FORMAT('%s as %s', column_name, column_name)
 END
 FROM
   `bq-info-schema-analysis.bqsap.INFORMATION_SCHEMA.COLUMNS`
 WHERE
   table_name = TBL.TABLE_NAME);
 
DECLARE
 QUERY_STR STRING DEFAULT FORMAT("""
   CREATE OR REPLACE VIEW `bq-info-schema-analysis.bqsap.%s_view` AS
   SELECT %s FROM `bq-info-schema-analysis.bqsap.%s`
   """,
   TBL.TABLE_NAME,ARRAY_TO_STRING(FIELDS_ARR, ","), TBL.TABLE_NAME);
EXECUTE IMMEDIATE
 QUERY_STR;
END
 ;
END
 FOR;

Query complete after 0.00s: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 399.84query/s]


""
